In [33]:
from langchain_community.tools import DuckDuckGoSearchRun,tavily_search
from langgraph.prebuilt import create_react_agent
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from langchain_openai import AzureChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from state import AgentState
from langgraph.types import Command
from langchain_core.messages import HumanMessage
from typing import Literal
from langgraph.graph import END
from prompt import make_system_prompt
from langchain.tools import tool
from dotenv import load_dotenv
import os

load_dotenv()

tavily_key = os.getenv("TAVILY_APIKEY")

token_provider = get_bearer_token_provider(DefaultAzureCredential(),"https://cognitiveservices.azure.com/.default")
model= AzureChatOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider= token_provider,
    model= "gpt-4o-mini"
)

# Create the agent
memory = MemorySaver()
# Use the agent
#config = {"configurable": {"thread_id": "abc123"}}


search_tool = tavily_search.TavilySearchResults( tavily_api_key=tavily_key)



search_agent = create_react_agent(
    model, 
    [search_tool],
    #checkpointer=memory,
    prompt="You are a research-only agent. When you find data, output it and say **handover to chart agent**.",
    )

In [34]:
events =search_agent.stream(
    {
        "messages": [
            HumanMessage(content="First, get the UK's GDP over the past 5 years, then make a line chart of it. Once you make the chart, finish.")
        ],
        "sender":"user"
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 5}
)
for s in events:
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xiaMqXUsQsQX3drjChGQsyde', 'function': {'arguments': '{"query":"UK GDP over the past 5 years"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 132, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-BmvEpA3B7oYkv5Wer83XR7dl745Lx', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 's